In [ ]:
import ee
import geemap

ee.Initialize(project="ee-nathanielpaulrobinson")
print(ee.__version__)

In [10]:
# Load Private Assets
asset_path = "projects/ee-nathanielpaulrobinson/assets/NTRI-data/"
ntri_villages = ee.FeatureCollection(asset_path + "NTRI-villages")
ntri_extent = ee.FeatureCollection(asset_path + "NTRI-boundary")

# Load Public Assets
mod44b = ee.ImageCollection("MODIS/006/MOD44B")
hansen_gfc = ee.Image("UMD/hansen/global_forest_change_2023_v1_11")
protected_areas = ee.FeatureCollection("WCMC/WDPA/current/polygons")

In [ ]:
# Village Selection
assessment_village_names = [
    "Eleng'ata Dapash",
    "Engaruka chini",
    "Kakoi",
    "Katikati",
    "Kimana",
    "Kitwai A",
    "Losirwa",
    "Matale A",
    "Nadonjukin",
    "Ngoswak",
    "Olchoroonyokie",
    "Sangaiwe",
]

assessment_villages = ntri_villages.filter(
    ee.Filter.inList("Village", assessment_village_names)
)

In [43]:
# MOD44B Processing
nonvegetated_band = "Percent_NonVegetated"
pre_start_date = "2010"
pre_end_date = "2015"
post_start_date = "2016"
post_end_date = "2021"

mod44b_nonvegetated = mod44b.select(nonvegetated_band)

pre_nonvegetated = (
    mod44b_nonvegetated.filterDate(pre_start_date, pre_end_date)
    .mean()
    .rename("pre_nonvegetated")
)
post_nonvegetated = (
    mod44b_nonvegetated.filterDate(post_start_date, post_end_date)
    .mean()
    .rename("post_nonvegetated")
)

In [32]:
# Create Pixel Area Image
land_mask = hansen_gfc.select("datamask").eq(1)
area = ee.Image.pixelArea().multiply(0.0001).updateMask(land_mask)

In [41]:
# Calculate NonVegetated Area
nonvegetated = pre_nonvegetated.addBands(post_nonvegetated).divide(100).multiply(area)

bare_stats = nonvegetated.reduceRegions(
    collection=assessment_villages, scale=250, crs="EPSG:4326", reducer=ee.Reducer.sum()
).select(["Village", "pre_nonvegetated", "post_nonvegetated"])

{'type': 'Feature', 'geometry': {'type': 'Polygon', 'coordinates': [[[35.913483921704035, -3.752373670448004], [35.91442034510058, -3.7553657078198848], [35.91531660782111, -3.756560737778917], [35.91834439078525, -3.7592540585019214], [35.92268308176833, -3.7623576240276524], [35.931770716001374, -3.7699112975814932], [35.93397794975646, -3.771516590413923], [35.935511898748956, -3.772863226219661], [35.936408170509736, -3.7743971643635987], [35.94010924496456, -3.777389195416373], [35.94089402951644, -3.7776879737023306], [35.94164315739458, -3.7779109607640344], [35.94283817400386, -3.779707984825554], [35.94388609413068, -3.781166064461882], [35.945495797193296, -3.7820980356540628], [35.94665519172938, -3.7822853462518764], [35.94934401048359, -3.7828471677460374], [35.951404101845455, -3.784117999060004], [35.953794235295234, -3.7854646604464337], [35.9555153987128, -3.7861736838779514], [35.9566390863641, -3.786137958990168], [35.95891773429046, -3.7856876265449944], [35.9623958

In [34]:
Map = geemap.Map()
Map.centerObject(ntri_extent)

Map.add_layer(nonvegetated)
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…